In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.8.0


In [ ]:
from google.colab import files
uploaded_file = files.upload()

In [ ]:
dataset = pd.read_csv('dataset3.csv')
dataset.head()

,Dates,Hours,Category,X,Y,Label
0,31/03/2015 23:00:00,23,ASSAULT,106.779610,-6.250459,1
1,31/03/2015 22:30:00,22,VEHICLE THEFT,106.888504,-6.201302,1
2,31/03/2015 22:30:00,22,VEHICLE THEFT,106.854430,-6.251243,1
3,31/03/2015 21:58:00,21,ASSAULT,106.805830,-6.127431,1
4,31/03/2015 21:45:00,21,VEHICLE THEFT,106.842910,-6.228551,1


In [ ]:
dataset.shape

(14402, 6)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14402 entries, 0 to 14401
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Dates     14402 non-null  object 
 1   Hours     14402 non-null  int64  
 2   Category  14402 non-null  object 
 3   X         14402 non-null  float64
 4   Y         14402 non-null  float64
 5   Label     14402 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 675.2+ KB


In [ ]:
dataset.drop(['Dates', 'Category'], axis=1, inplace=True)

In [ ]:
xs = dataset.iloc[:, 0:-1]
ys = dataset.iloc[:, -1]

In [ ]:
dataset.describe()

,Hours,X,Y,Label
count,14402.000000,14402.000000,14402.000000,14402.000000
mean,11.436884,106.842759,-6.196510,0.175739
std,6.976909,0.056754,0.044963,0.380612
min,0.000000,106.723350,-6.285473,0.000000
25%,6.000000,106.798924,-6.239150,0.000000
50%,11.000000,106.840000,-6.200000,0.000000
75%,18.000000,106.900000,-6.160000,0.000000
max,23.000000,106.941390,-6.120000,1.000000


In [ ]:
mean = [x for x in np.mean(xs)]
variance = [x for x in np.var(xs)]

In [ ]:
model = keras.Sequential([
      layers.InputLayer(input_shape=(3,)),
      layers.Normalization(axis=-1, mean=mean, variance=variance),
      layers.Dense(32, input_shape=(3,), activation='relu'),
      layers.Dense(2, activation='softmax')
  ])

model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['Accuracy'])

In [ ]:
#normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
#normalizer.adapt(np.array(xs))

In [ ]:
'''print(normalizer.mean.numpy())
first = np.array(xs[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())'''

In [ ]:
'''def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(32, activation='relu'),
      layers.Dense(2, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['Accuracy'])
  return model'''

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_2 (Normalizat  (None, 3)                0         
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 32)                128       
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 194
Trainable params: 194
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%%time
history = model.fit(xs, ys, verbose=1, validation_split=0.2, epochs=10)

Epoch 1/10
361/361 [==============================] - 1s 3ms/step - loss: 0.4751 - Accuracy: 0.7767 - val_loss: 0.1646 - val_Accuracy: 1.0000
Epoch 2/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4437 - Accuracy: 0.7957 - val_loss: 0.1565 - val_Accuracy: 0.9972
Epoch 3/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4362 - Accuracy: 0.8077 - val_loss: 0.1490 - val_Accuracy: 0.9969
Epoch 4/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4309 - Accuracy: 0.8142 - val_loss: 0.1482 - val_Accuracy: 0.9903
Epoch 5/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4259 - Accuracy: 0.8208 - val_loss: 0.1414 - val_Accuracy: 0.9889
Epoch 6/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4210 - Accuracy: 0.8287 - val_loss: 0.1740 - val_Accuracy: 0.9813
Epoch 7/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4165 - Accuracy: 0.8360 - val_loss: 0.1662 - val_Accuracy: 0.9823
Epoch 

In [ ]:
input_data = [2, 106.815, -6.16388]

prediction = model.predict([input_data])

print(prediction.flatten()[1])
print(np.argmax(prediction))

0.1951505
0


In [ ]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

In [ ]:
import pathlib

tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

2992

In [ ]:
model.save('my_model.h5')